### EDA (Exploratory Data Analysis) & Data Cleaning

In [63]:

##import das bibliotecas e adequando colunas, linhas e formato de números

from google.cloud import bigquery
from dotenv import load_dotenv
import pandas as pd
import pandas_gbq as gbq
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import re

# Carrega o .env
load_dotenv()

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")

# Verificar se a configuração está correta
print("Credenciais do BigQuery:", os.getenv("GOOGLE_APPLICATION_CREDENTIALS"))

# Verifica se a variável está configurada
print(os.getenv("GOOGLE_APPLICATION_CREDENTIALS"))


# Configurar Pandas para exibir todas as colunas e todas as linhas completas
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 1000)


pd.options.display.float_format = '{:.2f}'.format

Credenciais do BigQuery: C:\Temp\desafiolh-445818-3cb0f62cb9ef.json
C:\Temp\desafiolh-445818-3cb0f62cb9ef.json


In [ ]:
# Configurar o cliente do BigQuery
client = bigquery.Client()

# Nome do dataset e tabela
dataset_id = 'raw_data'

In [ ]:
# Listar tabelas no dataset
tables = client.list_tables('raw_data')
print("Tabelas disponíveis:")
for table in tables:
    print(table.table_id)




In [ ]:
query = f"SELECT * FROM `raw_data.humanresources_employee`"
data = client.query(query).result().to_dataframe()

# Expandir a coluna JSON
EDA_humanresources_employee = pd.json_normalize(data['data'])

# Exibir os dados expandidos
print(EDA_humanresources_employee.sample(n=50))


### Verificando valores nulos/em branco nos dados

In [ ]:
#valores nulos

EDA_humanresources_employee.isnull().sum()

In [ ]:
# dropar colunas vazias

In [ ]:
#total valores unicos de cada variável

valores_unicos = []

for i in EDA_humanresources_employee.columns[0:15].tolist():
    print(i, ':', len(EDA_humanresources_employee[i].astype(str).value_counts()))
    valores_unicos.append(len(EDA_humanresources_employee[i].astype(str).value_counts()))

In [ ]:
# Identificar duplicatas com base em 'businessentityid'
duplicatas = EDA_humanresources_employee[EDA_humanresources_employee.duplicated(subset=['businessentityid'], keep=False)]

# Verificar se existem duplicatas
if not duplicatas.empty:
    # Ordenar duplicatas por 'businessentityid' e 'modifieddate'
    duplicatas_ordenadas = duplicatas.sort_values(by=['businessentityid', 'modifieddate'])
    
    # Exibir duplicatas ordenadas
    print("Duplicatas ordenadas:")
    print(duplicatas_ordenadas)
else:
    print("Não foram encontradas duplicatas.")


In [ ]:
print(duplicatas_ordenadas.drop_duplicates())


In [ ]:
duplicados_businessentityid = EDA_humanresources_employee[EDA_humanresources_employee.duplicated(subset=['businessentityid'], keep=False)]

# Ordenar por 'businessentityid' para facilitar a análise
duplicados_ordenados = duplicados_businessentityid.sort_values(by=['businessentityid'])

# Exibir todas as linhas duplicadas
print(duplicados_ordenados)


In [ ]:
pd.set_option('display.max_rows', None)
contagem = EDA_humanresources_employee['businessentityid'].value_counts()

# Filtrar apenas os IDs que aparecem mais de uma vez
repetidos = contagem[contagem > 1]

# Exibir repetidos novamente
print(repetidos)

In [ ]:
#copia da humanresources_employee
raw_data_bkp = EDA_humanresources_employee.copy()

# Ordenar o DataFrame por 'businessentityid' e 'modifieddate'
EDA_humanresources_employee = EDA_humanresources_employee.sort_values(by=['businessentityid', 'modifieddate'])

# Remover duplicatas mantendo a última ocorrência com base em 'modifieddate'
EDA_humanresources_employee = EDA_humanresources_employee.drop_duplicates(subset=['businessentityid'], keep='last')

print(f"Linhas após remover duplicatas (baseando-se na última 'modifieddate'): {len(EDA_humanresources_employee)}")


In [ ]:
# Verificar informações do DataFrame
print(EDA_humanresources_employee.info())


In [ ]:
# Identificar colunas com datas
date_columns = ['birthdate', 'hiredate', 'modifieddate']

# Converter as colunas para datetime se ainda não estiverem
for col in date_columns:
    EDA_humanresources_employee[col] = pd.to_datetime(EDA_humanresources_employee[col], errors='coerce')

# Criar uma cópia do DataFrame para exibição formatada
formatted_data = EDA_humanresources_employee.copy()

# Formatar todas as colunas de datas para exibição
for col in date_columns:
    formatted_data[col] = EDA_humanresources_employee[col].dt.strftime('%Y-%m-%d %H:%M:%S')

# Exibir o DataFrame formatado
print(formatted_data.head())

# Verificar os tipos originais permanecem datetime64[ns]
print("\nTipos originais das colunas no DataFrame principal:")
print(EDA_humanresources_employee[date_columns].dtypes)


In [ ]:
# Padronizar textos em jobtitle e gender
EDA_humanresources_employee['jobtitle'] = EDA_humanresources_employee['jobtitle'].str.strip().str.title()
EDA_humanresources_employee['gender'] = EDA_humanresources_employee['gender'].str.strip().str.upper()

# Verificar valores únicos para garantir a padronização
print("Valores únicos em 'jobtitle':", EDA_humanresources_employee['jobtitle'].unique())
print("Valores únicos em 'gender':", EDA_humanresources_employee['gender'].unique())


In [ ]:
# Identificar colunas numéricas para análise 
numeric_columns = ['sickleavehours', 'vacationhours']

# Exibir estatísticas descritivas
print(EDA_humanresources_employee[numeric_columns].describe())

# Calcular limites para outliers (IQR - Intervalo Interquartil)
for col in numeric_columns:
    q1 = EDA_humanresources_employee[col].quantile(0.25)
    q3 = EDA_humanresources_employee[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    # Exibir os limites
    print(f"\nColuna: {col}")
    print(f"Limite inferior: {lower_bound}, Limite superior: {upper_bound}")
    
    # Filtrar outliers
    outliers = EDA_humanresources_employee[(EDA_humanresources_employee[col] < lower_bound) | (EDA_humanresources_employee[col] > upper_bound)]
    print(f"Outliers detectados ({len(outliers)}):")
    print(outliers[[col]])


In [ ]:

# Criar a coluna 'hire_year' com base em 'hiredate'
EDA_humanresources_employee['hire_year'] = EDA_humanresources_employee['hiredate'].dt.year


#verificando outros dados para detectar outliers
anos_contratacao = [int(ano) for ano in EDA_humanresources_employee['hire_year'].unique()]
anos_contratacao.sort()
print(anos_contratacao)

print("Valores únicos em 'gender':", EDA_humanresources_employee['gender'].unique())
print("Valores únicos em 'salariedflag':", EDA_humanresources_employee['salariedflag'].unique())


In [ ]:
# Verificar valores ausentes
missing = EDA_humanresources_employee.isnull().sum()
print("Valores ausentes por coluna:")
print(missing)

# Tratar colunas críticas
if missing['hiredate'] > 0:
    print("Tratar valores ausentes em 'hiredate' (decisão: remover ou imputar)")

if missing['jobtitle'] > 0:
    print("Tratar valores ausentes em 'jobtitle' (decisão: remover ou imputar)")

if missing['businessentityid'] > 0:
    print("Erro crítico: 'businessentityid' não pode ter valores ausentes!")


In [ ]:
# Verificar linhas com 'modifieddate' ausente
missing_modifieddate = EDA_humanresources_employee[EDA_humanresources_employee['modifieddate'].isnull()]
print("Linhas com 'modifieddate' ausente:")
print(missing_modifieddate)


In [ ]:
# Preencher 'modifieddate' ausente ou igual a 'hiredate', pois pode ser a ultima data de modificação no sistema.
EDA_humanresources_employee.loc[EDA_humanresources_employee['modifieddate'].isnull() | (EDA_humanresources_employee['modifieddate'] == pd.Timestamp('1900-01-01')), 'modifieddate'] = EDA_humanresources_employee['hiredate']

# Exibir as linhas ajustadas
print("Linhas onde 'modifieddate' foi ajustado para 'hiredate':")
print(EDA_humanresources_employee.loc[EDA_humanresources_employee['modifieddate'] == EDA_humanresources_employee['hiredate']])


In [ ]:
# Verificar unicidade de 'businessentityid'
is_unique = EDA_humanresources_employee['businessentityid'].is_unique
print(f"'businessentityid' é único? {is_unique}")


In [ ]:
# Definir regex para validar números (exemplo: apenas dígitos, 9 caracteres)
# acrescentei para ver se tinha um padrão, mas não tem
regex = r'^\d{9}$'

# Verificar valores inválidos
invalid_nationalid = EDA_humanresources_employee[~EDA_humanresources_employee['nationalidnumber'].astype(str).str.match(regex)]
print(f"Valores inválidos em 'nationalidnumber':\n{invalid_nationalid['nationalidnumber']}")


In [ ]:
is_unique = EDA_humanresources_employee['nationalidnumber'].is_unique
print(f"'nationalidnumber' é único? {is_unique}")


In [ ]:
# Criar um backup do DataFrame tratado
EDA_humanresources_employee_bkp_v2 = EDA_humanresources_employee.copy()

# Verificar o tamanho do backup e as primeiras linhas
print(f"Backup criado com {len(EDA_humanresources_employee_bkp_v2)} linhas.")
print(EDA_humanresources_employee_bkp_v2.head())


In [ ]:
# Verificar e documentar colunas existentes
print("Colunas mantidas no dataset:", EDA_humanresources_employee.columns.tolist())


In [ ]:
for col in EDA_humanresources_employee.columns:
    print(f"Valores únicos em '{col}':", EDA_humanresources_employee[col].unique()[:3])  # Limitar a exibição a 10 valores



In [ ]:
# Listar colunas binárias esperadas
binary_columns = ['currentflag', 'salariedflag']

# Verificar valores únicos em colunas binárias
for col in binary_columns:
    unique_values = EDA_humanresources_employee[col].unique()
    print(f"Valores únicos em '{col}': {unique_values}")

   



In [ ]:
# Contar valores em 'currentflag' e 'salariedflag'
print("Distribuição de 'currentflag':")
print(EDA_humanresources_employee['currentflag'].value_counts())

print("\nDistribuição de 'salariedflag':")
print(EDA_humanresources_employee['salariedflag'].value_counts())


#se vale a pena deletar ou não a coluna currentflag, já que só tem 1 valor e é true ?!

In [ ]:
# 1. Verificar se todos os funcionários ativos têm currentflag = True, pois deveria ser false = demitido/desligado
print("Funcionários ativos errados:", EDA_humanresources_employee[EDA_humanresources_employee['currentflag'] != True])

# 2. Validar datas
print("Contratações futuras:", EDA_humanresources_employee[EDA_humanresources_employee['hiredate'] > pd.Timestamp.now()])
print("Modifieddate antes de hiredate:", EDA_humanresources_employee[EDA_humanresources_employee['modifieddate'] < EDA_humanresources_employee['hiredate']])

# 3. Verificar unicidade de identificadores
print("Duplicados em 'businessentityid':", EDA_humanresources_employee['businessentityid'].duplicated().sum())
print("Duplicados em 'nationalidnumber':", EDA_humanresources_employee['nationalidnumber'].duplicated().sum())



In [ ]:
# Definir Dataset e Nome da Tabela
dataset_id = 'raw_data_cleaned'  # Substitua pelo nome do seu Dataset
table_id = 'humanresources_employee'   # Substitua pelo nome da sua Tabela
full_table_id = f"{client.project}.{dataset_id}.{table_id}"  # Nome completo da tabela

# Exportar para o BigQuery
EDA_humanresources_employee.to_gbq(full_table_id, if_exists='replace', credentials=client._credentials)



## ESTATÍSTICA DESCRITIVA

In [ ]:
# Selecionar colunas relevantes para análise descritiva
cols_para_analise = ['sickleavehours', 'vacationhours', 'salariedflag']

# Garantir que as datas estejam no formato correto
EDA_humanresources_employee['hire_year'] = pd.to_datetime(EDA_humanresources_employee['hiredate']).dt.year

# Adicionar a nova coluna à lista
cols_para_analise.append('hire_year')

# Gerar estatísticas descritivas
analise_descritiva = EDA_humanresources_employee[cols_para_analise].describe(include='all')

# Substituir NaN por '-'
analise_descritiva = analise_descritiva.fillna('-')

print(analise_descritiva)
